## Smali & Manifest File Analysis

This notebook will try to analyze a directory and its SMALI and MANIFEST files. The analysis will find and print sensitive methods and packages used by the program. 

In [1]:
from os import walk
import pandas as pd

## Initially, we declare the lists, and read csvs into the dataframe.

In [2]:
methods_to_find = []
perms_to_find = []
found_perms = []
found_lines = []

#read file using pandas
df = pd.read_csv('functions.csv', engine='python')
methods_to_find = df.functions
df = pd.read_csv('perms.csv', engine='python')
perms_to_find   = df.perms

In [3]:
#perms_to_find.iloc[-1]  LAST ELEMENT
frame = [perms_to_find, methods_to_find]
mergedfind = pd.concat(frame)
df = pd.DataFrame(columns=mergedfind)
df.to_csv(path_or_buf="analysis.csv")

In [4]:
#list to be appended
newRow = [0] * mergedfind.size

df

,READ_CALENDAR,WRITE_CALENDAR,CAMERA,READ_CONTACTS,WRITE_CONTACTS,GET_ACCOUNTS,ACCESS_FINE_LOCATION,ACCESS_COARSE_LOCATION,RECORD_AUDIO,READ_PHONE_STATE,...,getExternalCacheDir,shareIntent,readPhoneNumber,FileInputStream,getSharedPreferences,telephony,getMessageBody,getDisplayOriginatingAddress,wallpaper,crypto


In [11]:
def findFunctionsInDir(dir, list=0, verbose = True):
    #for each directory, and files
    for dirpath, dirnames, files in walk(dir):

        #skip the res folder, too many subfolders, slows.
        if('res' in dirpath): continue

        if(verbose == True): print(f'Found directory: {dirpath}')
        for file_name in files:

            #if it is a smali file
            if('.smali' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                with open(dirpath + "\\" + file_name) as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in methods_to_find:
                            if(i in line and 'invoke' in line):
                                #print the harmful line

                                #df[methods_to_find.index(i)] = 1
                                #df.iloc[0, methods_to_find.index(i)] = 1
                                #initialize a whole row with 0s.

                                if(line.strip() not in found_lines):
                                    if(verbose == True): print(num, ": ", line)
                                    found_lines.append(line.strip())

            #if it is a manifest file
            if('.xml' in file_name and 'anifest' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                #I had to use latin encoding here or it would simply not work.
                with open(dirpath + "\\" + file_name, encoding="latin-1") as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in perms_to_find:
                            if(i in line and ('<uses-permission android:name=' in line)
                                         or  ('<action android:name="android.app.action') in line):
                                
                                #problem line right here
                                #newRow[perms_to_find.index(i)] = 1
                                #df.iloc[0, perms_to_find.index(i)] += 1
                                
                                #print the harmful line
                                if(line.strip() not in found_perms):
                                    if(verbose == True): print(num, ": ", line)
                                    found_perms.append(line.strip())

#

In [6]:
#list to be appended
#newList = [0] * mergedfind.size

#append list
df.loc[len(df.index)] = newRow

#clear dataframe
#df = df.iloc[0:0]

#check if empty
#df.empty

In [7]:
perms_to_find.iloc[0]
#whyy

'READ_CALENDAR'

In [12]:
#directory = "C:\\Users\\vinic\\Desktop\\apktool\\simp"\
directory = "C:\\Users\\Vin\\Desktop\\apktool\\Sample1-WannaLocker\\sample1"

findFunctionsInDir(dir=directory, verbose=True)

Found directory: C:\Users\Vin\Desktop\apktool\Sample1-WannaLocker\sample1
AndroidManifest.xml
2 :      <uses-permission android:name="android.permission.SET_WALLPAPER"/>

5 :      <uses-permission android:name="android.permission.READ_PHONE_STATE"/>

6 :      <uses-permission android:name="android.permission.GET_TASKS"/>

7 :      <uses-permission android:name="android.permission.WRITE_EXTERNAL_STORAGE"/>

14 :      <uses-permission android:name="android.permission.MOUNT_UNMOUNT_FILESYSTEMS"/>

15 :      <uses-permission android:name="android.permission.READ_EXTERNAL_STORAGE"/>

Found directory: C:\Users\Vin\Desktop\apktool\Sample1-WannaLocker\sample1\assets
Found directory: C:\Users\Vin\Desktop\apktool\Sample1-WannaLocker\sample1\original
AndroidManifest.xml
Found directory: C:\Users\Vin\Desktop\apktool\Sample1-WannaLocker\sample1\original\META-INF
Found directory: C:\Users\Vin\Desktop\apktool\Sample1-WannaLocker\sample1\smali
Found directory: C:\Users\Vin\Desktop\apktool\Sample1-Wann

In [9]:
#pretty print the found lines
#print('\n'.join(found_lines))
df

,READ_CALENDAR,WRITE_CALENDAR,CAMERA,READ_CONTACTS,WRITE_CONTACTS,GET_ACCOUNTS,ACCESS_FINE_LOCATION,ACCESS_COARSE_LOCATION,RECORD_AUDIO,READ_PHONE_STATE,...,getExternalCacheDir,shareIntent,readPhoneNumber,FileInputStream,getSharedPreferences,telephony,getMessageBody,getDisplayOriginatingAddress,wallpaper,crypto
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#pretty print found permissions and actions
print('\n'.join(found_perms))

<uses-permission android:name="android.permission.SET_WALLPAPER"/>
<uses-permission android:name="android.permission.READ_PHONE_STATE"/>
<uses-permission android:name="android.permission.GET_TASKS"/>
<uses-permission android:name="android.permission.WRITE_EXTERNAL_STORAGE"/>
<uses-permission android:name="android.permission.MOUNT_UNMOUNT_FILESYSTEMS"/>
<uses-permission android:name="android.permission.READ_EXTERNAL_STORAGE"/>


In [ ]:
#TODO: split 2 search functionalities
#TODO: Add a function counter
#TODO: add a GUI
#TODO: add a way to select path
#TODO: Increase the search list
#TODO: Print results in a better way
#TODO: output to a file. (csv)